In [1]:
import pandas as pd, skill_cat, multiprocessing.dummy
from jobs_skills_weights import *
import synonyms

In [2]:
skill_cat._refresh()
synonyms._refresh()

# What is the goal here?
For each job, create a dict from (a skill name that job wants) to (a float indicating the importance of that skill)

In [3]:
# What fraction of skill weight comes from being listed in a job description?
# 1-list_weight gives the weight of being in a category
list_weight = 4/5.0 

In [4]:
jobs = get_jobs(bookmarked=False)
raw_job_details = get_raw_job_details(jobs)

In [5]:
skill_cat._refresh()
synonyms._refresh()

In [6]:
job_skill_text_data = pd.concat(
    {key: extract_skills_data(value) for key, value in raw_job_details.items()},
    names=["id"]
).groupby(level=[0,2,3]).sum().query("`skill`.isin(@skill_cat.skill_to_categories.keys())").sort_index()

jst_counts = job_skill_text_data.pop("count")
job_skill_text_data["share of job"] = jst_counts / jst_counts.groupby(level=0).sum()

In [7]:
jobs_categories_skills_data = pd.DataFrame([{
        "id":job_id,
        "category":category,
        "skill":skill_name,
        "share of job":len(categories_skill_is_in), # This skill will appear in `n` categories, so we'll divide it by `n` to keep it conserved.
    } for job_id in raw_job_details.keys()
        for skill_name, categories_skill_is_in in skill_cat.skill_to_categories.items()
            for category in categories_skill_is_in
]).set_index(["id", "category", "skill"])

In [8]:
jcst_data = (job_skill_text_data / jobs_categories_skills_data).reset_index().fillna({"share of job":0}) # jcst_data => jobs_categories_skills_text_data
jcst_data["skill text"].fillna(jcst_data["skill"], inplace=True)
jcst_data = jcst_data.set_index(["id","category","skill","skill text"]).sort_index()

In [10]:
from skill_weights import category_biases

jcst_data["share of job"] = (jcst_data["share of job"] * (1-category_biases.sum())) + (category_biases/jcst_data.groupby(level=["id","category"]).size())

In [12]:
jcst_data

share of job
id                                   category skill            skill text                    
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science computer science      0.013333
                                              experimentation  experimentation       0.000000
                                              research         research              0.000000
                                     Admin    automation       automating            0.026667
                                              aws              aws                   0.000000
...                                                                                       ...
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix             unix                  0.000000
                                     Visual   graphic design   graphic design        0.000000
                                              usability        usability             0.000000
                                     WebDev   javascript       javascript            0.000000
                                              web development  web development       0.000000

[13156 rows x 1 columns]

In [13]:
jcst_wc = (jcst_data*list_weight + (jcst_data*(1-list_weight)).groupby(level=[0,1]).mean())

In [14]:
jcst_wc

share of job
id                                   category skill            skill text                    
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science computer science      0.011556
                                              experimentation  experimentation       0.000889
                                              research         research              0.000889
                                     Admin    automation       automating            0.023018
                                              aws              aws                   0.001684
...                                                                                       ...
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix             unix                  0.000436
                                     Visual   graphic design   graphic design        0.000000
                                              usability        usability             0.000000
                                     WebDev   javascript       javascript            0.000000
                                              web development  web development       0.000000

[13156 rows x 1 columns]

In [15]:
jst_wc = jcst_wc.groupby(level=[0,2,3]).sum()

In [16]:
jst_wc.head(50)

share of job
id                                   skill                skill text                        
0124e218-2d33-41e8-907b-5228ea386455 agile                agile                     0.007759
                                     algebra              algebra                   0.003111
                                     algorithms           algorithms                0.004137
                                     analysis             analysis                  0.001867
                                     apache tinkerpop     apache tinkerpop          0.000667
                                     api design           api design                0.001026
                                     automation           automating                0.023018
                                     aws                  aws                       0.003018
                                     aws neptune          aws neptune               0.000667
                                     azure databricks     azure databricks          0.001333
                                     bash                 bash                      0.011745
                                     big data             big data                  0.003200
                                     bioinformatics       bioinformatics            0.000000
                                     budget               budget                    0.000381
                                     build tools          build tools               0.001212
                                     c                    c                         0.010533
                                     c++                  c++                       0.010533
                                     cancer               cancer                    0.000000
                                     ci/cd                ci/cd                     0.001684
                                     cloud computing      cloud computing           0.001684
                                     coding               coding                    0.011559
                                     collaboration        collaboration             0.007378
                                     communication        communication             0.028711
                                                          communication skills      0.028711
                                     computer engineering computer engineering      0.001212
                                     computer science     computer science          0.023248
                                     containerization     devops                    0.045563
                                                          docker                    0.024230
                                                          kubernetes                0.024230
                                     curious              curious                   0.007378
                                     data                 data                      0.067867
                                     data driven          data driven               0.007378
                                     data engineering     data engineering          0.001867
                                     data manipulation    data manipulation         0.001867
                                     data pipelines       pipelines                 0.023200
                                     data science         data science              0.023200
                                     debugging            debugging                 0.001026
                                     deep learning        deep learning             0.003111
                                     documentation        documentation             0.001026
                                     end to end           end to end                0.023018
                                     experimentation      experimentation           0.000889
                                     feature engineering  feature engineering    